In [24]:
import pandas as pd
#import datetime
from datetime import datetime, timedelta

In [69]:
#load apor CSVs, rename columns and format to dates
variable_rate_df = pd.read_csv('yieldtableadjustable.csv')
variable_rate_df['Years to First Adjustment'] = variable_rate_df['Years to First Adjustment'].str.strip(' ')
variable_rate_df.rename(columns = {'Years to First Adjustment':'date'}, inplace=True)
variable_rate_df['date'] = pd.to_datetime(variable_rate_df['date'])

fixed_rate_df = pd.read_csv('yieldtablefixed.csv')
fixed_rate_df['Term of Loan in Years'] = fixed_rate_df['Term of Loan in Years'].str.strip(' ')
fixed_rate_df.rename(columns = {'Term of Loan in Years':'date'},inplace=True)
fixed_rate_df['date'] = pd.to_datetime(fixed_rate_df['date'])


In [70]:
fixed_rate_df.head()


,date,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,2000-01-03,8.64,7.92,7.62,7.62,7.40,7.40,7.47,7.47,7.27,...,8.17,8.17,8.17,8.17,8.17,8.17,8.17,8.17,8.17,8.17
1,2000-01-10,8.49,7.89,7.63,7.63,7.44,7.44,7.49,7.49,7.34,...,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26,8.26
2,2000-01-17,8.69,8.02,7.72,7.72,7.52,7.52,7.53,7.53,7.39,...,8.29,8.29,8.29,8.29,8.29,8.29,8.29,8.29,8.29,8.29
3,2000-01-24,8.83,8.08,7.76,7.76,7.53,7.53,7.53,7.53,7.41,...,8.37,8.37,8.37,8.37,8.37,8.37,8.37,8.37,8.37,8.37
4,2000-01-31,8.54,7.94,7.69,7.69,7.53,7.53,7.54,7.54,7.43,...,8.36,8.36,8.36,8.36,8.36,8.36,8.36,8.36,8.36,8.36


In [71]:
variable_rate_df.tail()

,date,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
864,2016-07-25,3.28,3.25,3.21,3.21,3.15,3.15,3.21,3.21,3.29,...,3.29,3.29,3.29,3.29,3.29,3.29,3.29,3.29,3.29,3.29
865,2016-08-01,3.29,3.26,3.22,3.22,3.15,3.15,3.21,3.21,3.28,...,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.28
866,2016-08-08,3.25,3.21,3.18,3.18,3.11,3.11,3.18,3.18,3.26,...,3.26,3.26,3.26,3.26,3.26,3.26,3.26,3.26,3.26,3.26
867,2016-08-15,3.31,3.27,3.23,3.23,3.15,3.15,3.21,3.21,3.27,...,3.27,3.27,3.27,3.27,3.27,3.27,3.27,3.27,3.27,3.27
868,2016-08-22,3.32,3.28,3.23,3.23,3.15,3.15,3.21,3.21,3.26,...,3.26,3.26,3.26,3.26,3.26,3.26,3.26,3.26,3.26,3.26


In [99]:
class calculator(object):
    
    """calculator object that returns the rate spread variable required for HMDA submissions"""
    def __init__(self, fixed_data, variable_data):
        #load apor data
        self.fixed_data = fixed_data
        self.variable_data = variable_data
    
    
    def get_fixed_apor(self, term=None, lock_date=None):
        """returns the fixed APOR value associated with the term and lock in date"""
        #raise value error if any input is none or incorrectly formatted or out of bounds
        if term is None or lock_date is None:
            raise ValueError("All inputs must be populated")
        elif int(term) < 1 or int(term) > 50:
            raise ValueError("Term must be between 1 and 50")
        elif lock_date < self.fixed_data.date.min().date():
            raise ValueError("Lock-in date must be after 2000-03-01")
        elif lock_date > self.fixed_data.date.max().date():
            raise ValueError("Lock-in date must be before"+ str(self.fixed_data.date.max().date))
        while len(self.fixed_data[term][self.fixed_data.date == lock_date])<1:
            print(lock_date - timedelta(days=1))
            lock_date = lock_date - timedelta(days=1)

        if len(self.fixed_data[term][self.fixed_data.date == lock_date]) == 1:
            result = (self.fixed_data[term][self.fixed_data.date == lock_date])
            return result
    
    
    def get_variable_apor(self, term=None, lock_date=None):
        """returns the fixed APOR value associated with the term and lock in date"""
        #raise value error if any input is none or incorrectly formatted or out of bounds
        #print(type(lock_date), 'lock date in var apor')
        if term is None or lock_date is None:
            raise ValueError("All inputs must be populated")
        elif int(term) < 1 or int(term) > 50:
            raise ValueError("Term must be between 1 and 50")
        elif lock_date < self.variable_data.date.min().date():
            raise ValueError("Lock-in date must be after 2000-03-01")
        elif lock_date > self.variable_data.date.max().date():
            raise ValueError("Lock-in date must be before" + str(self.variable_data.date.max())

        while len(self.variable_data[term][self.variable_data['date'] == lock_date]) <1:
            print(lock_date - timedelta(days=1))
            lock_date = lock_date - timedelta(days=1)

        if len(self.variable_data[term][self.variable_data.date == lock_date]) == 1:
            result = (self.variable_data[term][self.variable_data.date == lock_date])
            return result
        
    
    def format_date(self, lock_date):
        """takes a string input and returns a date object"""
        date = datetime.strptime(lock_date, '%Y-%m-%d').date()
        return date
    
    
    def calc_rate_spread(self, apr=None, amort=None, lock_date=None, term=None, lien=None):
        """calculates the rates spread based on amortization type, term, lien status and date of lock in"""
        #raise value error if any inputs are None, incorrectly formatted or out of bounds
        lock_date = self.format_date(lock_date)
        if apr is None or amort is None or lock_date is None or term is None or lien is None:
            raise ValueError("All inputs must be populated")
        elif amort == 'fixed':
            print('getting fixed rate data')
            apor = float(self.get_fixed_apor(term, lock_date))
        elif amort == 'variable':
            print('getting var rate data')
            apor = float(self.get_variable_apor(term, lock_date))
        else:
            raise ValueError("amortization type must be fixed or variable")
        
        if lien == '1' and apr-apor < 1.5:
            return 'NA'
        elif lien == '2' and apr-apor <3.5:
            return 'NA'
        elif int(lien) > 2:
            return 'NA'
        else:
            print(apr-apor)
            return apr-apor
        #use JSON format to accept multiple lines input_key: list of inputs
        #FIXME do returns need to account for other HMDA fields like action or property type?

SyntaxError: invalid syntax (<ipython-input-99-d1d6ff787236>, line 43)

In [100]:
calc = calculator(fixed_rate_df, variable_rate_df)

In [101]:
calc.calc_rate_spread(apr=10.05, amort='fixed', lock_date='2020-08-21', term='30', lien='1')


getting fixed rate data


ValueError: Lock-in date must be before<built-in method date of Timestamp object at 0x10df8aa98>

In [102]:
calc.calc_rate_spread(apr=9.01, amort='variable', lock_date='2020-08-22', term='5', lien='1')

getting var rate data


ValueError: Lock-in date must be before2016-08-22